<a href="https://colab.research.google.com/github/maxrgnt/pythdc2-project2/blob/master/EDA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Panel Data
import pandas as pd
# System folders
import os
from pathlib import Path
# Visualization
import plotly.graph_objs as go
import seaborn as sns
%matplotlib inline

In [0]:
def convertToDateTime(year):
  return pd.to_datetime(year, format='%Y')

In [19]:
# read in the data
# url='https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/borderCrossing.csv'
# url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/pctChangeGDP.csv'
url = 'https://raw.githubusercontent.com/maxrgnt/pythdc2-project2/master/data/unemployment.csv'
df = pd.read_csv(url)
df.sample(1)

,State,Year,Population,LaborForce,Employed,Unemployed,Abrv
43,Vermont,1978,359833.0,239606.0,226182.0,13424.0,VT


In [0]:
# df['Year'] = pd.to_datetime(df['Year'], format='%Y')

In [21]:
df.dtypes

State          object
Year            int64
Population    float64
LaborForce    float64
Employed      float64
Unemployed    float64
Abrv           object
dtype: object

In [22]:
# Handle the Date column
df['newDate'] = pd.to_datetime(df['Date'])

KeyError: ignored

In [0]:
# Inspect the date column
df['Year'] = df['newDate'].dt.year
df['Month'] = df['newDate'].dt.month
# How many years of data available?
maxYear = df['newDate'].dt.year.max()
minYear = df['newDate'].dt.year.min()
print(f'From {minYear} to {maxYear} for total of {maxYear-minYear} year(s) of data.')

From 1996 to 2019 for total of 23 year(s) of data.


In [33]:
df2 = df.groupby(['State','Year','Abrv'])['Employed','Unemployed'].sum().reset_index()

,State,Year,Abrv,Employed,Unemployed
0,Alaska,1976,AL,151190.0,12380.0
1,Alaska,1977,AL,155948.0,17065.0
2,Alaska,1978,AL,162730.0,19381.0
3,Alaska,1979,AL,166730.0,17077.0
4,Alaska,1980,AL,169397.0,18008.0
...,...,...,...,...,...
640,Washington,2014,WA,3277540.0,213947.0
641,Washington,2015,WA,3349995.0,200455.0
642,Washington,2016,WA,3451378.0,192320.0
643,Washington,2017,WA,3543665.0,175235.0


In [0]:
def calcPctChange(df, col2calc):
  for y in df[col2calc].unique():
    print(y)
    blah = 1.0
    if y != df['Year'].min():
      t0 = df.loc[(df['Year'] == y-1) & (df['State'] == x),'Value'].tolist()[0]
      t1 = df.loc[(df['Year'] == y  ois) & (df['State'] == x),'Value'].tolist()[0]
      df.loc[df['Year'] == y, 'pctChange'] = (1-t1/t0)*100
  print('hi')

In [34]:
# calcPctChange(df2, 'Year')
for state in df2['State'].unique():
  print(state)

Alaska
Arizona
California
Idaho
Maine
Michigan
Minnesota
Montana
New Mexico
New York
North Dakota
Ohio
Texas
Vermont
Washington


In [0]:

for y in list(brdr['Year'].unique()):
  blah = 1.0
  if y != brdr['Year'].min():
    t0 = brdr.loc[brdr['Year'] == y-1,'immigrants'].tolist()[0]
    t1 = brdr.loc[brdr['Year'] == y,'immigrants'].tolist()[0]
    brdr.loc[brdr['Year'] == y, 'pct'] = (1-t1/t0)*100
brdr = brdr[brdr['Year'].astype(int)>brdr['Year'].min()]